In [1]:
currentDirContent = !ls
currentDirContent
from myFirebase import MyFirebaseService

myfirebase = MyFirebaseService(certPath='./firebase.json'
                 , collectionName='tweets_health')



In [2]:
import re

In [4]:
%%time

# allTweet = myfirebase.getCollectionRef().get()
tmp=[]

# for t in allTweet:
for t in  myfirebase.getCollectionRef().stream():
    # if 'text' in t.to_dict().keys(): tmp.append(t.to_dict()['text'])
    td = t.to_dict()
    if 'text' in td.keys(): tmp.append(td['text'])
    



CPU times: user 51.9 s, sys: 5.59 s, total: 57.5 s
Wall time: 53.4 s


In [5]:
%%time
tweetList = []
re_ausvote_auspol = re.compile('ausvot\\w*|auspol\\w*', re.I)
for text in tmp:
    if re.search(re_ausvote_auspol,text ) is not None:
        tweetList.append(text)

CPU times: user 524 ms, sys: 0 ns, total: 524 ms
Wall time: 526 ms


In [6]:
len(tweetList)

10211

In [7]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Sun Jun 12 06:52:31 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   36C    P0    28W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [31]:


!pip install bertopic
!pip install -U sentence-transformers

from bertopic import BERTopic

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [29]:
!pip install -U spacy
!python -m spacy download en_core_web_sm

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 12.8 MB 4.1 MB/s 
  Attempting uninstall: en-core-web-sm
    Found existing installation: en-core-web-sm 2.2.5
    Uninstalling en-core-web-sm-2.2.5:
      Successfully uninstalled en-core-web-sm-2.2.5
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


In [11]:
# docs = fetch_20newsgroups(subset='all',  remove=('headers', 'footers', 'quotes'))['data']
# docs = fetch_20newsgroups(subset='all')['data']
docs = tweetList

In [54]:

from sentence_transformers import SentenceTransformer
embedding_model = SentenceTransformer('distilbert-base-nli-mean-tokens')
# embeddings = embedding_model.encode(docs, show_progress_bar=True)

bertopic = BERTopic(embedding_model=embedding_model, calculate_probabilities=True, verbose=True, n_gram_range=(1,2))

# model = BERTopic()

In [32]:
removeKw =   ['ausvot','auspol']
re_removeKw = re.compile('|'.join([f'{kw}\\w*' for kw in removeKw]), re.I )
testtttt = 'ausvoting, ausvotes, auspolice, whatever it is'
re.sub(re_removeKw, " ", testtttt)

' ,  ,  , whatever it is'

In [90]:
import string
import spacy
import re

removeKw =   ['ausvot','auspol']
re_removeKw = re.compile('|'.join([f'{kw}\\w*' for kw in removeKw]) , re.I)

# re_nonChars = re.compile(r'[^a-zA-Z]+', re.I)
re_nonChars = re.compile(r'[!\"#$%&()*+/:;<=>?@[\]^{|}~]+', re.I)
re_link = re.compile(r'https*://\S+', re.I)
re_whitespace = re.compile(r'\s+', re.I)

# customPunctuation = f"[{string.punctuation.replace('.','')}]+"
# re_punctuation = re.compile(r"[!\"#$%&'()*+,-./:;<=>?@[\]^_`{|}~]+", re.I)

stopwords = spacy.load('en_core_web_sm').Defaults.stop_words


def cleaning(input, lemmaModel = None):
    input = input.lower()
    input = re.sub(re_removeKw, " ", input)
    input = re.sub(re_link, " ", input)
    input = re.sub(re_nonChars, " ", input)
    input = re.sub(re_whitespace, " ", input)
    
    tmp = []
    tmp2 = []
    for w in input.split():
        if w.lower() not in stopwords: tmp.append(w)

    if lemmaModel:
        for token in lemmaModel(' '.join(tmp)):
            tmp2.append(token.lemma_)
        res = ' '.join(tmp2)
    else: res = ' '.join(tmp)
    return res


def cleaningStringList(stringList, cleanFunc=cleaning):
    result = []
    nlp = spacy.load('en_core_web_sm', disable = ['parser','ner'])
    for s in stringList:
        result.append(cleanFunc(s, nlp))
    return result




# cleaningStringList(sub_docs[:3])


In [60]:
tttt = ['https://google.com','http://google.com, auspol 123312 auspol2022 ausvotes2022']
cleaningStringList(tttt)

['', '123312']

In [91]:
sub_docs = cleaningStringList(docs)

In [92]:
%%time
topics, probs = bertopic.fit_transform(sub_docs)

Batches:   0%|          | 0/320 [00:00<?, ?it/s]

2022-06-12 08:11:22,743 - BERTopic - Transformed documents to Embeddings
2022-06-12 08:11:41,915 - BERTopic - Reduced dimensionality
2022-06-12 08:11:43,490 - BERTopic - Clustered reduced embeddings


CPU times: user 46.1 s, sys: 605 ms, total: 46.7 s
Wall time: 35.6 s


In [93]:
topics, probs

([0,
  4,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  -1,
  0,
  0,
  -1,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  -1,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  -1,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  -1,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  10,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  -1,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0

In [94]:
bertopic.topics[-1]

[('woman', 0.016218782914018744),
 ('be', 0.012123709927425882),
 ('amp', 0.011122743199361997),
 ('morrison', 0.011014354267386084),
 ('australia', 0.009963830576616293),
 ('need', 0.009621977106382936),
 ('say', 0.009130421430251418),
 ('not', 0.00898332492311171),
 ('liberal', 0.008857702907795875),
 ('minister', 0.008780693974590375)]

In [95]:
bertopic.get_topic_info()

,Topic,Count,Name
0,0,9604,0_not_labor_election_be
1,-1,356,-1_woman_be_amp_morrison
2,1,71,1_lol collective_kid west_west lol_collective
3,2,23,2_completely_doubt government_if doubt_people ...
4,3,23,3_french_french president_president_submarine ...
5,4,22,4_lnpdeathcult_federalicac aspengate_dom_dom p...
6,5,21,5_queensland town_town_queensland_biloela
7,6,20,6_church_horizon_horizon church_local church
8,7,18,7_soccer_play_play soccer_child
9,8,16,8_ausgov_region_world_ukraine


In [107]:
bertopic.topics

{-1: [('woman', 0.016218782914018744),
  ('be', 0.012123709927425882),
  ('amp', 0.011122743199361997),
  ('morrison', 0.011014354267386084),
  ('australia', 0.009963830576616293),
  ('need', 0.009621977106382936),
  ('say', 0.009130421430251418),
  ('not', 0.00898332492311171),
  ('liberal', 0.008857702907795875),
  ('minister', 0.008780693974590375)],
 0: [('not', 0.014039451244633627),
  ('labor', 0.012484866693515305),
  ('election', 0.012297080243077184),
  ('be', 0.012208645328153911),
  ('morrison', 0.011991401795627023),
  ('amp', 0.011495690619088793),
  ('australia', 0.011070923677458051),
  ('vote', 0.010039240138424214),
  ('it', 0.009534526756500314),
  ('government', 0.009369577736297366)],
 1: [('lol collective', 1.4406690680060816),
  ('kid west', 1.4406690680060816),
  ('west lol', 1.4406690680060816),
  ('collective', 1.0637383251680583),
  ('lol', 0.9457485513299537),
  ('west', 0.9139185043225424),
  ('kid', 0.7848061750388359),
  ('', 1e-05),
  ('', 1e-05),
  ('', 

In [105]:
# bertopic.visualize_barchart(height=400, width=400, n_words=10, top_n_topics=16)
bertopic.visualize_barchart(height=400, width=400, n_words=10)

In [106]:
bertopic.get_topic(-1)

[('woman', 0.016218782914018744),
 ('be', 0.012123709927425882),
 ('amp', 0.011122743199361997),
 ('morrison', 0.011014354267386084),
 ('australia', 0.009963830576616293),
 ('need', 0.009621977106382936),
 ('say', 0.009130421430251418),
 ('not', 0.00898332492311171),
 ('liberal', 0.008857702907795875),
 ('minister', 0.008780693974590375)]

In [98]:
bertopic.get_topic_info()[:20]

,Topic,Count,Name
0,0,9604,0_not_labor_election_be
1,-1,356,-1_woman_be_amp_morrison
2,1,71,1_lol collective_kid west_west lol_collective
3,2,23,2_completely_doubt government_if doubt_people ...
4,3,23,3_french_french president_president_submarine ...
5,4,22,4_lnpdeathcult_federalicac aspengate_dom_dom p...
6,5,21,5_queensland town_town_queensland_biloela
7,6,20,6_church_horizon_horizon church_local church
8,7,18,7_soccer_play_play soccer_child
9,8,16,8_ausgov_region_world_ukraine


In [99]:
# bertopic.visualize_topics(top_n_topics=10)
bertopic.visualize_topics()

In [100]:
bertopic.visualize_hierarchy(top_n_topics=50)

In [101]:
bertopic_small = BERTopic(embedding_model=embedding_model, calculate_probabilities=True, verbose=True, n_gram_range=(1,2), nr_topics=10)
_ = bertopic_small.fit_transform(sub_docs)
bertopic_small.get_topic_info()

Batches:   0%|          | 0/320 [00:00<?, ?it/s]

2022-06-12 08:12:12,148 - BERTopic - Transformed documents to Embeddings
2022-06-12 08:12:30,310 - BERTopic - Reduced dimensionality
2022-06-12 08:12:36,744 - BERTopic - Clustered reduced embeddings
2022-06-12 08:12:42,784 - BERTopic - Reduced number of topics from 100 to 11


,Topic,Count,Name
0,-1,7741,-1_labor_election_morrison_be
1,0,441,0_million_wage_000_year
2,1,427,1_good_time_exciting time_be
3,2,416,2_australia_australian_completely_not
4,3,320,3_not_do_do not_will
5,4,222,4_liberal_party_liberal party_liberalaus
6,5,169,5_bad_be_morrison_he
7,6,147,6_election_final_week_day
8,7,147,7_australia_australian_albomp_well
9,8,97,8_health_medicine_medicinesaus_aus


In [102]:
bertopic_small.visualize_hierarchy()

In [103]:
bertopic_small.topics

{-1: [('labor', 0.01373814350887329),
  ('election', 0.013626783529501063),
  ('morrison', 0.013092258554260276),
  ('be', 0.012633909302554954),
  ('amp', 0.012486832579703228),
  ('not', 0.011743031801245857),
  ('vote', 0.011340742293914179),
  ('minister', 0.010171206174525228),
  ('australia', 0.010052694126766746),
  ('government', 0.009565203735708199)],
 0: [('million', 0.029225666469818043),
  ('wage', 0.027821585023632488),
  ('000', 0.02727153892032826),
  ('year', 0.020857929876590128),
  ('billion', 0.02032514799742829),
  ('cost', 0.019445475340127343),
  ('rise', 0.018838311887854856),
  ('price', 0.01609401481905408),
  ('pay', 0.015961920598469525),
  ('government', 0.01556430506046106)],
 1: [('good', 0.02713033621773579),
  ('time', 0.024854826090523453),
  ('exciting time', 0.02112453036543373),
  ('be', 0.02043097252814758),
  ('exciting', 0.019678574726056598),
  ('question', 0.01913762228588169),
  ('think', 0.016738371948977034),
  ('scomo', 0.016586947652725677

In [104]:
nlp = spacy.load('en_core_web_sm', disable = ['parser','ner'])
a = 'scomo australian australians australia work works worked working ausvotes2022'

for token in nlp(a):
    print(token.lemma_)

scomo
australian
australian
australia
work
work
work
work
ausvotes2022
